# Naive Simple MLP

In [ ]:
# ================= QUIET HEADER (경고 최소화; 맨 위에 두기) =================
import sys, types, warnings, os
try:
    import tqdm as _tqdm
    _auto = types.ModuleType("tqdm.auto"); _auto.tqdm = _tqdm.tqdm
    sys.modules["tqdm.auto"] = _auto
except Exception:
    pass
from sklearn.exceptions import DataConversionWarning, ConvergenceWarning
warnings.filterwarnings("ignore", category=DataConversionWarning)
warnings.filterwarnings("ignore", category=ConvergenceWarning)
warnings.filterwarnings("ignore", message=".*IProgress not found.*")
warnings.filterwarnings("ignore", message=".*Maximum iterations.*hasn't converged.*")
# ===========================================================================

# ================= USER CONFIG =============================================
DATA_FILE     = "dataset.csv"               # index='Time'
Y_COLS        = ["xr_2","xr_3","xr_5","xr_7","xr_10"]

SLOPE_PREFIX  = "s_"                        # 예: s_2, s_3, ...
MACRO_PREFIX  = "F"                         # 예: FIP, FUMD, F... 등 'F'로 시작
FWD_PREFIX    = "fwd_"                      # 예: fwd_1, fwd_2, ...

PERIOD        = ["197108", "202312"]
BURN_IN_END   = "199001"
HORIZON       = 12
SHOW_PROGRESS = True

RUN_RIDGE     = True
RUN_MLP       = True

# 자주 바꿀만한 최소 파라미터/그리드만 노출
RIDGE_PARAMS  = dict(random_state=0)        # alpha는 그리드에서 튜닝
RIDGE_CV      = {"mode":"tscv","n_splits":10,"grid":{"alpha":[1e-3,1e-2,1e-1,1,10,100]}}

MLP_PARAMS    = dict(                        # 기본값 (필요 시만 수정)
    random_state=0,
    max_iter=2000,
    early_stopping=True,
    learning_rate_init=1e-3,
    tol=1e-5,
)
MLP_CV        = {"mode":"tscv","n_splits":8,
                 "grid":{"hidden_layer_sizes":[(16)], "alpha":[1e3}}
# ===========================================================================

import numpy as np
import pandas as pd
from rolling_framework import ExpandingRunner, make_strategy

# ----------------------- helpers -----------------------
def read_df(path: str) -> pd.DataFrame:
    df = pd.read_csv(path, index_col="Time"); df.index = df.index.astype(str)
    return df

def cols_by_prefix(df: pd.DataFrame, prefix: str) -> pd.DataFrame:
    return df.loc[:, df.columns.str.startswith(prefix)]

def make_features(df: pd.DataFrame, *, use_slope: bool, use_macro: bool, use_fwd: bool) -> pd.DataFrame:
    parts = []
    if use_slope:
        parts.append(cols_by_prefix(df, SLOPE_PREFIX))
    if use_macro:
        parts.append(cols_by_prefix(df, MACRO_PREFIX))
    if use_fwd:
        parts.append(cols_by_prefix(df, FWD_PREFIX))
    if not parts:
        raise ValueError("No features selected.")
    # 중복 컬럼 방지 + 원본 순서 보존
    X = pd.concat(parts, axis=1)
    X = X.loc[:, ~X.columns.duplicated(keep="first")]
    return X

def run_experiment(model_name: str, params: dict, cv: dict, X: pd.DataFrame, y: pd.DataFrame,
                   tag: str, scale: bool = True):
    strat = make_strategy(model_name, target_cols=Y_COLS, params=params, scale=scale, cv=cv)
    runner = ExpandingRunner(X=X, y=y, strategy=strat,
                             period=PERIOD, burn_in_end=BURN_IN_END, horizon=HORIZON)
    runner.fit_walk(progress=SHOW_PROGRESS, desc=f"{model_name} on {tag}")
    r2_naive = runner.R2OOS(baseline="naive").round(4)
    r2_cond  = runner.R2OOS(baseline="condmean").round(4)
    print(f"\n=== {model_name} on {tag} ===")
    print("R2OOS vs naive:\n",    r2_naive)
    print("R2OOS vs condmean:\n", r2_cond)
    return runner

# ----------------------- load & split -------------------
df = read_df(DATA_FILE)
y  = df[Y_COLS].copy()

# 두 실험용 특징세트
X_slope_macro     = make_features(df, use_slope=True, use_macro=True, use_fwd=False)
X_slope_macro_fwd = make_features(df, use_slope=True, use_macro=True, use_fwd=True)

# ----------------------- RUN: Ridge ---------------------
if RUN_RIDGE:
    _ = run_experiment("Ridge", RIDGE_PARAMS, RIDGE_CV, X_slope_macro, y, "Slope + Macro", scale=True)
    _ = run_experiment("Ridge", RIDGE_PARAMS, RIDGE_CV, X_slope_macro_fwd, y, "Slope + Macro + Fwd", scale=True)

# ----------------------- RUN: MLP -----------------------
if RUN_MLP:
    _ = run_experiment("MLP", MLP_PARAMS, MLP_CV, X_slope_macro, y, "Slope + Macro", scale=True)
    _ = run_experiment("MLP", MLP_PARAMS, MLP_CV, X_slope_macro_fwd, y, "Slope + Macro + Fwd", scale=True)